<a href="https://colab.research.google.com/github/DownOnCoffee/GenAI_Learning/blob/main/huggingfacetut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!nvidia-smi

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification,pipeline

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
inferenceModel = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizerModel = AutoTokenizer.from_pretrained(model_name)

model = pipeline(task="sentiment-analysis",model=inferenceModel,tokenizer=tokenizerModel)
text = 'i love watching old school movies.'
print(model(text))

Tokenization (in depth)

In [ ]:
tokenizer_model = AutoTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer_model.tokenize(text)
print(tokens)

In [ ]:
token_ids = tokenizer_model.convert_tokens_to_ids(tokens)
print(token_ids)

In [ ]:
print(tokenizer_model.decode(token_ids))

Fine Tuning Imdb dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset('imdb')
dataset
dataset['train'][3]

Padding makes short sequences longer by adding special [PAD] tokens.
With batched=True, Hugging Face passes a list of texts, not one text
Truncation shortens long sequences by cutting off extra tokens.

In [ ]:
from transformers import AutoTokenizer
tmodel = AutoTokenizer.from_pretrained('bert-base-uncased')
def tokenizeContent(data):
  text = tmodel(data['text'],padding='max_length',truncation=True)
  return text
res = dataset.map(tokenizeContent,batched=True)
res




In [ ]:
from transformers import pipeline,AutoModelForSequenceClassification, TrainingArguments
train_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    eval_strategy ="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=1,
    weight_decay=0.01,
)
training_args

In [ ]:
from transformers import Trainer
TrainingModel=Trainer(
    model=train_model,
    args=training_args,
    train_dataset=res['train'],
    eval_dataset=res['test']
)

In [ ]:
TrainingModel.train()